# Camera calibration

In [1]:
import matplotlib
matplotlib.use('TkAgg')  # non-notebook window that supports point clicking
from machinevisiontoolbox import Image, CentralCamera
import matplotlib.pyplot as plt

from spatialmath import SE3
from spatialmath import base
np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})


Create a set of 3D coordinates that reflect the markers on the calibration target

![calibration](calibration-fixture.png)

In [2]:
cm = 0.01

P = np.array([
    [ 0,  -12.2, 12.2],
    [ 0,   -6.2, 12.2],
    [ 0,  -12.2,  6.2],
    [ 0,   -6.2,  6.2],
    [ 6.2,  0,   12.2],
    [12.2,  0,   12.2],
    [ 6.2,  0,    6.2],
    [12.2,  0,    6.2]
]).T * cm

We can establish a camera position

In [3]:
campos = np.r_[1, -1, 0.3]

create a camera pose that is lookign toward the origin. a-vector (z-axis) point to origin, o-vector (y-axis) points downward

In [4]:
T = SE3(campos) * SE3.OA([0,0,-1], -campos)
T

SE3:   0.707107    0.146735   -0.691714    1            
       0.707107   -0.146735    0.691714   -1            
       0          -0.978232   -0.207514    0.3          
       0           0           0           1            
    

Create a camera model

In [5]:
cam = CentralCamera(imagesize=(1280,1024), f=0.015, pose=T)
print(cam)

principal point not specified,                    setting it to centre of image plane
           Name: MVTB camera [CentralCamera]
     pixel size: 1e-05 x 1e-05
     image size: 1280.0 x 1024.0
           pose: t = 1, -1, 0.3; rpy/zyx = -102°, 0°, 45°
   principal pt: (640.0, 512.0)
   focal length: (0.015, 0.015)



The camera intrinsic matrix is

In [6]:
cam.K

array([[    1500,        0,      640],
       [       0,     1500,      512],
       [       0,        0,        1]])

and the camera (calibration) matrix is

In [7]:
cam.C

array([[     618,     1503,   -132.8,    925.2],
       [  -134.1,    134.1,    -1574,    740.2],
       [ -0.6917,   0.6917,  -0.2075,    1.446]])

Now we can project the points to the image plane 

In [8]:
p = cam.project(P)
p

array([[   543.1,    592.3,      544,    592.7,    687.7,    736.9,    687.3,      736],
       [   398.1,    391.9,    464.4,    456.4,    391.9,    398.1,    456.4,    464.4]])

In [9]:
Cd = CentralCamera.camcal(P, p)

residual is 2.8e-11 px


In [10]:
cam2 = CentralCamera.invcamcal(Cd)

[[       1        0        0]
 [       0        1        0]
 [       0        0        1]]


In [11]:
image = Image('calib-image.png', grey=True)
image.disp()

reading image  (240, 320)
Colormap is  gray


<AxesSubplot:title={'center':'calib-image.png'}>

In [12]:
p = image.pickpoints(8)

In [13]:
p

array([[   80.16,    137.4,     80.9,    137.8,    224.6,    286.3,    223.8,      286],
       [    17.1,    32.44,    87.45,    91.94,    29.07,    9.992,    90.44,    84.45]])

In [14]:
C = CentralCamera.camcal(P, p)

residual is 0.632 px


In [15]:
C

array([[   132.9,    990.4,   -14.57,    180.4],
       [    -299,    252.1,   -853.4,    147.2],
       [  -2.543,    2.215, -0.08044,        1]])

In [16]:
cam2 = CentralCamera.invcamcal(C)

[[       1        0        0]
 [       0        1        0]
 [       0        0        1]]


In [17]:
type(cam2)

machinevisiontoolbox.Camera.CentralCamera

In [18]:
print(cam2)

           Name: MVTB camera [CentralCamera]
     pixel size: 1e-05 x 1e-05
     image size: 500.0 x 500.0
           pose: t = -0.22, -0.00727, 0.202; rpy/zyx = -91.4°, 0.43°, 48.9°
   principal pt: (163.2248376019937, 121.94393441897947)
   focal length: (247.2551838563246, 247.2551838563246)

